<a href="https://colab.research.google.com/github/nanimo22/TIL/blob/master/yykim/yykim_LuhnSummarize_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd


In [8]:
#konlpy 설치
!pip install konlpy

     |████████████████████████████████| 19.4MB 2.7MB/s 
     |████████████████████████████████| 92kB 3.1MB/s 
     |████████████████████████████████| 3.8MB 33.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


## Luhn Summarize Class로 만들기 

In [18]:
class LuhnSummarize:
    def __init__(self, text, doc, keysentence_number=1, min=0.01, max=0.2):
    # text는 문서 전체, doc는 문서 내 문장을 각각 리스트화 한 이중 리스트
        self.keysentence_number = keysentence_number
        self.doc = doc
        self.min = min
        self.max = max
        self.sentence_score = []


    def _make_score(self):
        from konlpy.tag import Kkma
        kkma= Kkma()

        #전체 문서 토크나이징
        total_token = kkma.morphs(text)
        unique_token = np.unique(total_token)
        
        #{유니크 토큰: 토큰 등장 횟수}로 딕셔너리 정리        
        n_unique_token=[]
        for token in unique_token:
            tmp = total_token.count(token)
            n_unique_token.append(tmp)
        dict_unique_token = dict(zip(list(unique_token),n_unique_token))

        #문장별로 토크나이즈
        tokenized_sens = []
        for sens in doc:
            tokenized_sen = kkma.morphs(sens)
            tokenized_sens.append(tokenized_sen)

        # 각 문장의 토큰들을 해당 토큰의 빈도수/전체토큰수(score)로 치환
        import copy
        words_score = tokenized_sens.copy()

        for sen_idx, sen in enumerate(tokenized_sens):
            for morph_idx,morph in enumerate(sen):
                words_score[sen_idx][morph_idx] = dict_unique_token[morph]/len(total_token)

        # 중요 단어로 판단된 단어들의 문장 내 인덱스 확인
        keywords_idx=[]
        for sen_idx, sen in enumerate(words_score):
            keywords_idx_tmp = []
            for morph_idx,morph in enumerate(sen):
                if self.min < morph < self.max:
                    keywords_idx_tmp.append(morph_idx)
            keywords_idx.append(keywords_idx_tmp)

        for sens in keywords_idx:
            try:
                self.sentence_score.append(np.square(sens[-1]-sens[0]+1)/len(sens))
            except:
                self.sentence_score.append(0) #문장 내에 중요 단어가 1개 이하인 경우
        return self.sentence_score                       
    
    def luhn_summarize(self):
        self._make_score()
        sort = np.argsort(self.sentence_score)[::-1]
        top = sort[0:self.keysentence_number]

        keysentence_ls = []
        for i in top :
            keysentence_ls.append(self.doc[i])
        return keysentence_ls        

In [19]:
text="""
코로나19 대응 의료 인공지능 컨소시엄 출범 산학연병 인공지능 전문가 20여 명으로 구성, 기술‧응용 공동 연구
지난 8일, 창립준비 모임…미영상의학회 COVID-19 컨소시엄과도 협력

코로나19 바이러스의 세계적 감염 유행에 효과적으로 대응하기 위해 관련 전문가들이 모인 ‘코로나19 감염 대응을 위한 의료 인공지능 컨소시엄’이 9월 1일에 출범한다. 

연세대 의대, 한국전자통신원, 네이버 등 산학연병 20여 명이 참여하는 컨소시엄에서는 인공지능 기술과 응용 분야를 공동으로 연구하고 개발, 검증할 예정이다. 또 이 인공지능 기술을 적용하기 위해 필요한 데이터를 수집, 공유, 활용한다. 나아가 다양한 산학연병 전문가들이 국내외적으로 협력하고 공동으로 대응해 나갈 예정이다.
 
컨소시엄에는 15개 기관의 20여 명 전문가들이 참여했다. 연세대 의과대학에서는 박유랑(의생명시스템정보학), 김휘영(방사선의과학연구소), 심규원(신경외과), 염준섭(감염내과), 최병욱(영상의학과, 사진), 허진(영상의학과) 교수가 참여했다.

대학군에서는 권인호(동아대), 김남국(울산대), 김윤현(전남대), 김진영(계명대), 박상준(서울대), 신수용(성균관대), 정명진(성균관대), 진광남(서울대), 홍헬렌(서울여대) 교수가 , 산업군에서는 김기환(루닛), 이동훈(카카오), 정규환(뷰노), 최우식(딥노이드), 최정필(코어라인소프트), 하정우(네이버)가 기업을 대표해 동참했다.

이 밖에도 전종홍(한국전자통신연구원), 김경훈(정보통신정책연구원), 예종철(한국과학기술원) 교수도 포함됐다.
 
인공지능 컨소시엄은 지난 7월 8일에 창립 준비 위원 모임을 했고, 연세대 의대 영상의학과 최병욱 교수(사진)가 준비위원장으로 선출됐다.

컨소시엄은 북미영상의학회에서 시작한 COVID-19 RICORD 컨소시엄, 유럽연합에서 시작한 COVID-19 이미징 AI 이니셔티브 등과 함께 국제 협력 활동을 펼칠 계획이며, 코로나19 대응을 위한 AI 핵심표준 개발도 목표로 하고 있다. 아울러 대한의료인공지능학회, 대한영상의학회 등 국내 관련 학회와도 적극 협력할 계획이다.
 
한편 오는 7월 24일(금)에는 온라인 워크숍을 개최해 코로나 19 감염대응 의료 인공지능 관련 동향발표 및 연구개발 사례 소개가 있을 예정이다.
"""

In [12]:
#문서 문장으로 쪼개기 (해당 문서에서는 줄공백으로 구분)
doc = text.split('\n')

In [20]:
for sentence in doc:
    if sentence == '' or sentence == ' ':
        doc.remove(sentence)
print(doc)

print(len(doc))

['코로나19 대응 의료 인공지능 컨소시엄 출범 산학연병 인공지능 전문가 20여 명으로 구성, 기술‧응용 공동 연구', '지난 8일, 창립준비 모임…미영상의학회 COVID-19 컨소시엄과도 협력', '코로나19 바이러스의 세계적 감염 유행에 효과적으로 대응하기 위해 관련 전문가들이 모인 ‘코로나19 감염 대응을 위한 의료 인공지능 컨소시엄’이 9월 1일에 출범한다. ', '연세대 의대, 한국전자통신원, 네이버 등 산학연병 20여 명이 참여하는 컨소시엄에서는 인공지능 기술과 응용 분야를 공동으로 연구하고 개발, 검증할 예정이다. 또 이 인공지능 기술을 적용하기 위해 필요한 데이터를 수집, 공유, 활용한다. 나아가 다양한 산학연병 전문가들이 국내외적으로 협력하고 공동으로 대응해 나갈 예정이다.', '컨소시엄에는 15개 기관의 20여 명 전문가들이 참여했다. 연세대 의과대학에서는 박유랑(의생명시스템정보학), 김휘영(방사선의과학연구소), 심규원(신경외과), 염준섭(감염내과), 최병욱(영상의학과, 사진), 허진(영상의학과) 교수가 참여했다.', '대학군에서는 권인호(동아대), 김남국(울산대), 김윤현(전남대), 김진영(계명대), 박상준(서울대), 신수용(성균관대), 정명진(성균관대), 진광남(서울대), 홍헬렌(서울여대) 교수가 , 산업군에서는 김기환(루닛), 이동훈(카카오), 정규환(뷰노), 최우식(딥노이드), 최정필(코어라인소프트), 하정우(네이버)가 기업을 대표해 동참했다.', '이 밖에도 전종홍(한국전자통신연구원), 김경훈(정보통신정책연구원), 예종철(한국과학기술원) 교수도 포함됐다.', '인공지능 컨소시엄은 지난 7월 8일에 창립 준비 위원 모임을 했고, 연세대 의대 영상의학과 최병욱 교수(사진)가 준비위원장으로 선출됐다.', '컨소시엄은 북미영상의학회에서 시작한 COVID-19 RICORD 컨소시엄, 유럽연합에서 시작한 COVID-19 이미징 AI 이니셔티브 등과 함께 국제 협력 활동을 펼칠 계획이며, 코로나19 대응을 위한 AI 핵심표준 개발도 목표로 하고 

In [21]:
luhn_summarize = LuhnSummarize(text, doc, keysentence_number=3) 

In [22]:
luhn_summarize.luhn_summarize() #괄호의 단어점수 때문에 첫번째 문장이 잘못 선정됨(전처리 필요..)

['대학군에서는 권인호(동아대), 김남국(울산대), 김윤현(전남대), 김진영(계명대), 박상준(서울대), 신수용(성균관대), 정명진(성균관대), 진광남(서울대), 홍헬렌(서울여대) 교수가 , 산업군에서는 김기환(루닛), 이동훈(카카오), 정규환(뷰노), 최우식(딥노이드), 최정필(코어라인소프트), 하정우(네이버)가 기업을 대표해 동참했다.',
 '컨소시엄은 북미영상의학회에서 시작한 COVID-19 RICORD 컨소시엄, 유럽연합에서 시작한 COVID-19 이미징 AI 이니셔티브 등과 함께 국제 협력 활동을 펼칠 계획이며, 코로나19 대응을 위한 AI 핵심표준 개발도 목표로 하고 있다. 아울러 대한의료인공지능학회, 대한영상의학회 등 국내 관련 학회와도 적극 협력할 계획이다.',
 '연세대 의대, 한국전자통신원, 네이버 등 산학연병 20여 명이 참여하는 컨소시엄에서는 인공지능 기술과 응용 분야를 공동으로 연구하고 개발, 검증할 예정이다. 또 이 인공지능 기술을 적용하기 위해 필요한 데이터를 수집, 공유, 활용한다. 나아가 다양한 산학연병 전문가들이 국내외적으로 협력하고 공동으로 대응해 나갈 예정이다.']